In [1]:
# import the necessary python libraries:
from matplotlib import pyplot as plt # plotting
import matplotlib.gridspec as gridspec # more plotting 
import pylab as pl
import numpy as np
import math
import scipy
from functools import partial
import lmfit

#######################################
# DEFINE SEVERAL FUNCTIONS TO BE USED #
#######################################

# the inductive reactance of an inductor with inductance L
def XL(L, f):
    return 2 * math.pi * f * L

# the capacitive reactance of a capacitor with capacitance C
def XC(C, f):
    return 1 / (2 * math.pi * f * C)

# the impedance of the RLC circuit with total resistance RT, capacitance C, inductance L
def Zimp(f, RT, L, C):
    return math.sqrt(RT**2 + (XL(L, f)-XC(C, f))**2)

# the function that describes the voltage across the resistor (the measured experimental quantity)
def VR(f, R, VS, RL, L, C):
    RT = R+RL
    return VS * R/np.sqrt(RT**2 + (2 * math.pi * f * L - 1/(2 * math.pi * f * C))**2)

# negative the VR
def negVR(f, R, VS, RL, L, C):
    RT = R+RL
    return -VS * R/np.sqrt(RT**2 + (2 * math.pi * f * L - 1/(2 * math.pi * f * C))**2)

# the VR minus sqrt(2) to be used for the f1 and f2 frequencies (at half-power). 
def VRSqrt2(f, R, VS, RL, L, C):
    RT = R+RL
    f0=fres(L,C)
    return (1/Zimp(f0, RT, L, C))/(1/Zimp(f, RT, L, C)) - math.sqrt(2)

# the resonant frequency
def fres(L, C):
    return 1/(2 * math.pi * math.sqrt(L*C))

def round_sig(x, sig=2):
    if x == 0.:
        return 0.
    if math.isnan(x) is True:
        print('Warning, NaN!')
        return 0.
    return round(x, sig-int(math.floor(math.log10(abs(x))))-1)


In [20]:
##################
# the parameters
# R and RL in Ohm
# L in Henry
# C in Farad
#################
R = 500
RL = 6.39
C = 0.1E-6
VS = 3.0
L = 0.30

########################################################
# Your measurements go here: X is the frequency in Hz, #
# Y is the voltage across the resistor in Ohm          #
########################################################
X = np.array([410.2, 510.2, 610.2, 710.2, 810.2, 860.2, 910.2, 960.2, 1010, 1110, 1210, 1310, 1410])
Y = np.array([0.481, 0.671, 0.926, 1.249, 1.547, 1.622, 1.630, 1.581, 1.504, 1.319, 1.154, 1.018, 0.910])

# calculate the resonant frequency and print it
f0 = fres(L,C)
print('theoretical f0=', round_sig(f0,3), 'Hz')

#############################################
# Do fit of your data using the python "lmfit" package:
Xfit = np.arange(start=10, stop=2000.0, step=5.)
Y = np.array(Y)
print("Input data:")
print(X.flatten())
print(Y.flatten())
print('\n')
vmodel = lmfit.Model(VR)
params = vmodel.make_params()
#print(vmodel.param_names, vmodel.independent_vars)
params['R'].max = 1000
params['R'].min = 100
params['C'].min = 0.001E-6
params['C'].max = 0.10E-6
params['VS'].max = 5
params['VS'].min = 1.
params['L'].min = 0.01
params['L'].max = 1.0
params['RL'].min = 0.1
params['RL'].max = 20
result = vmodel.fit(Y.flatten(), params, f=X.flatten(), method='differential_evolution')
#print(result.fit_report()) # uncomment if you want to print the lmfit results 

#################################################################
# Use the fitted params to create an array for plotting the fit #
#################################################################
Rf, VSf, RLf, Lf, Cf = result.params['R'].value, result.params['VS'].value, result.params['RL'].value, result.params['L'].value, result.params['C'].value
# print the fitted parameters first:
print("R(fitted)=",round_sig(Rf,4), "Ohm")
print("V_S(fitted)=", round_sig(VSf,4), "V")
print("R_L(fitted)=", round_sig(RLf,4), "Ohm")
print("L(fitted)=",round_sig(Lf,4), 'H')
print("C(fitted)=",round_sig(Cf,4), "F")
Yfit = []
for j in range(0, len(Xfit)):
    Yfit.append(VR(Xfit[j],Rf, VSf, RLf, Lf, Cf))

####################
# get BW from fit: #
####################
# f1:
x01f = scipy.optimize.fsolve(VRSqrt2, 700, args=(Rf, VSf, RLf, Lf, Cf))
print("f1(from fit)=", round_sig(x01f[0],3), "Hz")
# f2:
x02f = scipy.optimize.fsolve(VRSqrt2, 1000, args=(Rf, VSf, RLf, Lf, Cf))
print("f2(from fit)=", round_sig(x02f[0],3), "Hz")
print('BW(from fit)=', round_sig(x02f[0]-x01f[0],3), "Hz")
# f0 from fit:
f0f = scipy.optimize.minimize(negVR, args=(Rf, VSf, RLf, Lf, Cf),x0=1000).x
print("f0(from fit)=",round_sig(f0f[0],3), "Hz")


theoretical f0= 919.0 Hz
Input data:
[ 410.2  510.2  610.2  710.2  810.2  860.2  910.2  960.2 1010.  1110.
 1210.  1310.  1410. ]
[0.481 0.671 0.926 1.249 1.547 1.622 1.63  1.581 1.504 1.319 1.154 1.018
 0.91 ]


R(fitted)= 999.7 Ohm
V_S(fitted)= 1.655 V
R_L(fitted)= 14.24 Ohm
L(fitted)= 0.308 H
C(fitted)= 9.99e-08 F
f1(from fit)= 682.0 Hz
f2(from fit)= 1210.0 Hz
BW(from fit)= 524.0 Hz
f0(from fit)= 907.0 Hz


In [21]:
###################################
# SET UP THE PLOTTING PARAMETERS  #
###################################
# plot name and labels
plot_type = 'RLC_resonance_fit' # the name of the plot
# the following labels are in LaTeX, but instead of a single slash, two "\\" are required.
ylab = '$V_R$ (V)' # the ylabel
xlab = 'Frequency (Hz)' # the x label
# log scale?
ylog = False # whether to plot y in log scale
xlog = False # whether to plot x in log scale

# construct the axes for the plot
# no need to modify this if you just need one plot
gs = gridspec.GridSpec(4, 4)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.grid(False)

# set the title:
ax.set_title("Voltage across resistor vs. AC Frequency")

#####################
# Graph the results #
#####################
# plot just points, no line
# set lw to a number different to zero if you want a line as well
plt.plot(X,Y, color='red', lw=0, marker='o', ms=5, label='')
plt.plot(Xfit,Yfit, color='green', lw=2, marker='', ms=5, label='')

# plot vertical lines at f1, f0, f2:
ax.vlines(x=x01f, ymin=0, ymax=VR(f0, Rf, VSf, RLf, Lf, Cf)/math.sqrt(2), linewidth=1, ls='--', color='red')
ax.vlines(x=f0, ymin=0, ymax=VSf, linewidth=2, ls='--', color='red')
ax.vlines(x=x02f, ymin=0, ymax=VR(f0, Rf, VSf, RLf, Lf, Cf)/math.sqrt(2), linewidth=1, ls='--', color='red')

#########################################
# set the ticks, labels and limits etc. #
#########################################

ax.set_ylabel(ylab, fontsize=20)
ax.set_xlabel(xlab, fontsize=20)
# choose x and y log scales
if ylog:
    ax.set_yscale('log')
else:
    ax.set_yscale('linear')
if xlog:
    ax.set_xscale('log')
else:
    ax.set_xscale('linear')

# set the limits on the x and y axes if required below:
xmin = 0.
xmax = 2000.0
ymin = 0.
ymax = 3.1
pl.xlim([xmin,xmax])
pl.ylim([ymin,ymax])

###################
# save the figure #
###################
print('saving the figure')
# save the figure in PDF format
infile = plot_type + '.dat'
print('---')
print('output in', infile.replace('.dat','.pdf'))
plt.savefig(infile.replace('.dat','.pdf'), bbox_inches='tight')
plt.close(fig)


saving the figure
---
output in RLC_resonance_fit.pdf
